In [1]:
# Directly copied this code from Lisowski's paper, entitled: 
# "Evaluation of Material Attractiveness to Non-state Actors of Various Nuclear Materials in Thorium Fuel Cycles"
# Running this in order test code and provide comparison point

#Modelling the 35-LEU/Th + ZrO2 central rod lattice concept
#This code is modified from the CANDU Bundle example provided at
#https://docs.openmc.org/en/latest/examples/candu.html
#And depletion was coded by following the example at

#https://docs.openmc.org/en/latest/examples/pincell-depletion.html

from math import pi, sin, cos
import numpy as np
import openmc
import openmc.deplete

fuel_materials = []
for i in range(0,35):
    fuel = openmc.Material(name='fuel');
    #The below ratios were calculated assuming 40%LEU-60%Th Oxide Fuel.
    fuel.add_nuclide('Th232', 0.60611,'ao')
    fuel.add_nuclide('U235',0.019695,'ao') #5% U-235 enrichment
    fuel.add_nuclide('U238',0.37420,'ao')
    fuel.add_element('O', 2.0)
    fuel.set_density('g/cm3', 9.7) #from Bromley paper
    fuel_materials.append(fuel)
    
#Center displacer rod of ZrO2
center = openmc.Material(name='rod')
center.add_element('Zr',1.0)
center.add_element('O',2.0)
center.set_density('g/cm3',4.3) #from Bromley paper

#Zircaloy-4 is approximated as just Zr
clad = openmc.Material(name='zircaloy')
clad.add_element('Zr', 1.0)
clad.set_density('g/cm3', 6.0) #default density from example CANDU Bundle

heavy_water = openmc.Material(name='heavy water')
heavy_water.add_nuclide('H2', 2.0)
heavy_water.add_nuclide('O16', 1.0)
heavy_water.add_s_alpha_beta('c_D_in_D2O')
heavy_water.set_density('g/cm3', 1.1) #default density from example CANDU Bundle

# Outer radius of fuel and clad
r_fuel_ex = 0.6122 #example fuel radius
r_clad_ex = 0.6540 #example clad radius
clad_thickness = r_clad_ex-r_fuel_ex #clad thickness NOT GIVEN in the Bromley 
# Thorium Fuel paper, so the value is calculated based on the example CANDU bundle
r_clad = 0.57 #outer radius of Thorium fuel element, given in Bromley paper
r_fuel = r_clad-clad_thickness

#Outer Radius of the center ZrO2 rod
r_center = 2.4

# Pressure tube and calendria radii - given in Bromley paper
pressure_tube_ir = 5.2
pressure_tube_or = 5.6
calendria_ir = 6.4
calendria_or = 6.6

# Radius to center of each ring of fuel pins; assumed based on the CANDU
# bundle example. This leads to overlapping in the OpenMC model, so the numbers
# were altered slightly.
#ring_radii = np.array([2.8755, 4.3305]) – original values in OpenMC CANDU Bundle example

#To avoid overlapping, I slightly changed the numbers below.
ring_radii = np.array([3.0755, 4.3305])

#Center Rod location
center_radius = np.array([0])

# These are the surfaces that will divide each of the rings
radial_surf = [openmc.ZCylinder(r=r) for r in
               (ring_radii[:-1] + ring_radii[1:])/2]

water_cells = []
for i in range(ring_radii.size):
    # Create annular region
    if i == 0:
        water_region = -radial_surf[i]
    elif i == ring_radii.size - 1:
        water_region = +radial_surf[i-1]
    else:
        water_region = +radial_surf[i-1] & -radial_surf[i]
        
    water_cells.append(openmc.Cell(fill=heavy_water, region=water_region))

#Also create a water region for the center tube
radial_surf_tube = openmc.ZCylinder(r=r_center+clad_thickness)

water_cell_tube = []
water_region_tube = -radial_surf_tube
water_cell_tube.append(openmc.Cell(fill=heavy_water,
region=water_region_tube))

plot_args = {'width': (2*calendria_or, 2*calendria_or)}
bundle_universe = openmc.Universe(cells=(water_cells))
#bundle_universe.add_cell(water_cell_tube)
#bundle_universe.plot(**plot_args)

#pin_cell universe for each fuel pin, divided between inner and outer pins for ease of iteration when creating fuel pins
pin_universes_inner = []
pin_universes_outer = []

fuel_cells = []
for i in range(0,14):
    surf_fuel = openmc.ZCylinder(r=r_fuel)
    fuel_cell = openmc.Cell(fill=fuel_materials[i], region=-surf_fuel)
    clad_cell = openmc.Cell(fill=clad, region=+surf_fuel)
    
    pin_universes_inner.append(openmc.Universe(cells=(fuel_cell, clad_cell)))
    fuel_cells.append(fuel_cell)
    
for i in range(14,35):
    surf_fuel = openmc.ZCylinder(r=r_fuel)
    fuel_cell = openmc.Cell(fill=fuel_materials[i], region=-surf_fuel)
    clad_cell = openmc.Cell(fill=clad, region=+surf_fuel)
    
    pin_universes_outer.append(openmc.Universe(cells=(fuel_cell, clad_cell)))
    fuel_cells.append(fuel_cell)
    
#Center rod universe
surf_center = openmc.ZCylinder(r=r_center+clad_thickness)

center_cell = openmc.Cell(fill=center, region=-surf_center)
center_tube_cell = openmc.Cell(fill=clad, region=+surf_center)

center_universe = openmc.Universe(cells=(center_cell, center_tube_cell))

#pin_universe.plot(**plot_args)

#center_universe.plot(**plot_args)

#Center tube creation

num_tubes = [1]
angles = [0]

for i, (r, n, a) in enumerate(zip(center_radius, num_tubes, angles)):
    tube_boundary = openmc.ZCylinder(x0=0, y0=0, r=r_center+clad_thickness)
    water_cells[i].region &= +tube_boundary
    
    tube = openmc.Cell(fill=center_universe, region=-tube_boundary)
    bundle_universe.add_cell(tube)
    
#bundle_universe.plot(**plot_args)

num_pins = [14, 21]
angles = [0, 0]

for i, (r, n, a) in enumerate(zip(ring_radii, num_pins, angles)):
    for j in range(n):
        # Determine location of center of pin
        theta = (a + j/n*360.) * pi/180.
        x = r*cos(theta)
        y = r*sin(theta)
        
        pin_boundary = openmc.ZCylinder(x0=x, y0=y, r=r_clad)
        water_cells[i].region &= +pin_boundary
        
        # Create each fuel pin -- note that we explicitly assign an ID so
        # that we can identify the pin later when looking at tallies
        if n == 14:
            pin = openmc.Cell(fill=pin_universes_inner[j], region=-pin_boundary)
            pin.translation = (x, y, 0)
            pin.id = (i + 1)*100 + j
            fuel_materials[j].id = pin.id #ID the fuel materials to match the pin ID
        if n == 21:
            pin = openmc.Cell(fill=pin_universes_outer[j], region=-
            pin_boundary)
            pin.translation = (x, y, 0)
            pin.id = (i + 1)*100 + j
            fuel_materials[14+j].id = pin.id #ID the fuel materials to match the pin ID
            bundle_universe.add_cell(pin)
            
        #bundle_universe.plot(**plot_args)

pt_inner = openmc.ZCylinder(r=pressure_tube_ir)
pt_outer = openmc.ZCylinder(r=pressure_tube_or)
calendria_inner = openmc.ZCylinder(r=calendria_ir)
calendria_outer = openmc.ZCylinder(r=calendria_or)

bundle = openmc.Cell(fill=bundle_universe, region=-pt_inner)
pressure_tube = openmc.Cell(fill=clad, region=+pt_inner & -pt_outer)
v1 = openmc.Cell(region=+pt_outer & -calendria_inner)
calendria = openmc.Cell(fill=clad, region=+calendria_inner & - calendria_outer)

box = openmc.rectangular_prism(width=28.575, height=28.575, boundary_type='reflective')

water_region = box & +calendria_outer
moderator = openmc.Cell(fill=heavy_water, region=water_region)

root_universe = openmc.Universe(cells=[bundle, pressure_tube, v1, calendria,moderator])

geom = openmc.Geometry(root_universe)
geom.export_to_xml()

mats = openmc.Materials(geom.get_all_materials().values())
mats.export_to_xml()

p = openmc.Plot.from_geometry(geom)
p.color_by = 'material'

p.colors = {
    clad: 'silver',
    center: 'gray',
    heavy_water: 'blue'
}
p.to_ipython_image()

settings = openmc.Settings()
settings.particles = 50000
settings.inactive = 50
settings.batches = 150

settings.source = openmc.Source(space=openmc.stats.Point())
settings.export_to_xml()

# tally is not manually determined by user in depletion

#Set up depletion material volume
#h_fuel = 49.5

for i in range(35):
    fuel_materials[i].volume = r_fuel ** 2 * pi
    #print(fuel_materials[0].volume)

# define time step over depletion
time_steps = [0.25, 0.25, 0.25, 0.25, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]

time_steps = [(0.25, 'MWd/kg'), (0.25, 'MWd/kg'), (0.25, 'MWd/kg'), (0.25, 'MWd/kg'),
 (1.0, 'MWd/kg'), (1.0, 'MWd/kg'), (1.0, 'MWd/kg'),
 (2.0, 'MWd/kg'), (2.0, 'MWd/kg'), (2.0, 'MWd/kg'), (2.0, 'MWd/kg'),
 (2.0, 'MWd/kg'), (2.0, 'MWd/kg'), (2.0, 'MWd/kg'), (2.0, 'MWd/kg'),
 (2.0, 'MWd/kg'), (2.0, 'MWd/kg'), (2.0, 'MWd/kg'), (2.0, 'MWd/kg'),
 (2.0, 'MWd/kg'), (2.0, 'MWd/kg'), (2.0, 'MWd/kg'), (2.0, 'MWd/kg'),
 (2.0, 'MWd/kg'), (2.0, 'MWd/kg'),
 (30.0,'d'), (30.0,'d'), (30.0,'d'), (30.0,'d'), (30.0,'d'), (30.0,'d'),
 (30.0,'d'), (30.0,'d'), (30.0,'d'), (30.0,'d'), (30.0,'d'), (30.0,'d'),
 (30.0,'d'), (30.0,'d'), (30.0,'d'), (30.0,'d'), (30.0,'d'), (30.0,'d'),
 (30.0,'d'), (30.0,'d'), (30.0,'d'), (30.0,'d'), (30.0,'d'), (30.0,'d')]

#define changing power over depletion and cooling
power_bundle = [0.00939e6, 0.00939e6, 0.00939e6, 0.00939e6,
                0.00939e6, 0.00939e6, 0.00939e6,
                0.00939e6, 0.00939e6, 0.00939e6, 0.00939e6,
                0.00939e6, 0.00939e6, 0.00939e6, 0.00939e6,
                0.00939e6, 0.00939e6, 0.00939e6, 0.00939e6,
                0.00939e6, 0.00939e6, 0.00939e6, 0.00939e6,
                0.00939e6, 0.00939e6,
                0, 0, 0, 0, 0, 0,
                0, 0, 0, 0, 0, 0,
                0, 0, 0, 0, 0, 0,
                0, 0, 0, 0, 0, 0] #W/cm

# Commented out depletion lines as it was throwing errors and I did not have time to figure out the fix
# set up depletion operator
# chain_file = './chain_endfb71_pwr.xml' # you can download it from https://openmc.org/depletion-chains/
# chain = openmc.deplete.Chain.from_xml(chain_file)

# op = openmc.deplete.Operator(geom, settings, chain_file)

# choose Predictor-Corrector scheme to run the depletion
# integrator = openmc.deplete.CECMIntegrator(op, time_steps, power_bundle)

# integrator.integrate()

In [2]:
# establish settings for criticality calculation
settings = openmc.Settings();
settings.run_mode = 'eigenvalue';
settings.particles = 5000;
settings.batches = 100;
settings.inactive = 10;

#box = openmc.stats.Box(lower_left = (-1.,-1.,-1.),
#                      upper_right = (1.,1.,1.),
#                      only_fissionable=True);
#src = openmc.Source(space=box);
point = openmc.stats.Point((0,0,0));
src = openmc.Source(space=point);

settings.source = src;
settings.export_to_xml();

In [3]:
openmc.run()

                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################

RuntimeError: Maximum number of lost particles has been reached. ERROR: Maximum number of lost particles has been reached. ERROR: Maximum number of lost particles has been reached.